In [ ]:
!sudo pip3 install tensorflow

    100% |████████████████████████████████| 109.2MB 13kB/s  eta 0:00:01
    100% |████████████████████████████████| 20.8MB 79kB/s  eta 0:00:01
    100% |████████████████████████████████| 51kB 4.3MB/s ta 0:00:011
    100% |████████████████████████████████| 1.3MB 1.3MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 7.2MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 3.6MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 5.3MB/s eta 0:00:01
    100% |████████████████████████████████| 13.4MB 114kB/s eta 0:00:01
    100% |████████████████████████████████| 3.2MB 538kB/s eta 0:00:01    74% |███████████████████████▊        | 2.3MB 16.5MB/s eta 0:00:01
    100% |████████████████████████████████| 491kB 3.3MB/s eta 0:00:01
    100% |████████████████████████████████| 2.9MB 585kB/s eta 0:00:01
    100% |████████████████████████████████| 307kB 3.6MB/s ta 0:00:011
    100% |████████████████████████████████| 92kB 6.2MB/s eta 0:00:01
  Running setup.py ins

In [ ]:
!sudo pip3 install h5py

In [ ]:
!sudo pip3 install numpy

In [ ]:
!sudo pip3 install pandas

In [ ]:
!sudo pip3 install --upgrade pip

In [ ]:
!sudo pip3 install opencv-python

In [ ]:
!sudo pip3 install matplotlib

In [ ]:
!sudo pip3 install keras==2.3.1

In [ ]:
!sudo pip3 install scikit-learn

In [ ]:
!sudo pip3 install ray

In [ ]:
!sudo pip3 install ray[tune]

In [ ]:
!sudo pip3 install tensorflow==2.2

In [ ]:
!sudo pip3 install keras

In [ ]:
!sudo pip3 install ray[rllib]

In [14]:
#!/usr/bin/env python3
import numpy as np 
import tensorflow as tf
import pandas as pd
import os
from cv2 import imread, createCLAHE 
import cv2
import pickle
from glob import glob
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from keras import backend as keras
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam 
from ray import tune

class UNet:
    def DataLoader(self):
        infile = open("/home/scitech/shared-data/mask_wf/op/data_split.pkl",'rb')
        new_dict = pickle.load(infile)
        infile.close()

        path = "/home/scitech/shared-data/mask_wf/op/"

        train_data = new_dict['train']
        valid_data = new_dict['valid']
        test_data = new_dict['test']

        X_train = [cv2.imread(os.path.join(path,i))[:,:,0] for i in train_data if 'mask' not in i]
        y_train = [cv2.imread(os.path.join(path,i))[:,:,0] for i in train_data if 'mask' in i]

        X_valid = [cv2.imread(os.path.join(path,i))[:,:,0] for i in valid_data if 'mask' not in i]
        y_valid = [cv2.imread(os.path.join(path,i))[:,:,0] for i in valid_data if 'mask' in i]

        X_test = [cv2.imread(os.path.join(path,i))[:,:,0] for i in test_data]

        dim = 256

        X_train = np.array(X_train).reshape((len(X_train),dim,dim,1))
        y_train = np.array(y_train).reshape((len(y_train),dim,dim,1))
        X_valid = np.array(X_valid).reshape((len(X_valid),dim,dim,1))
        y_valid = np.array(y_valid).reshape((len(y_valid),dim,dim,1))
        X_test = np.array(X_test).reshape((len(X_test),dim,dim,1))
        assert X_train.shape == y_train.shape
        assert X_valid.shape == y_valid.shape

        return X_train.astype(np.float32), y_train.astype(np.float32), X_valid.astype(np.float32), y_valid.astype(np.float32)
    
    def model(self, input_size=(256,256,1)):
        inputs = Input(input_size)
        conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
        conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
        conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
        conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
        conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
        conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

        up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
        conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
        conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

        up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
        conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
        conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

        up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
        conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
        conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

        up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
        conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
        conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

        conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

        return Model(inputs=[inputs], outputs=[conv10])

def dice_coef(y_true, y_pred):
        y_true_f = keras.flatten(y_true)
        y_pred_f = keras.flatten(y_pred)
        intersection = keras.sum(y_true_f * y_pred_f)
        return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)
    
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


class TuneReporterCallback(Callback):
    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()

    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        tune.report(keras_info=logs, mean_accuracy=logs.get("accuracy"), mean_loss=logs.get("loss"))

def tune_unet(config):
    unet = UNet()
    model = unet.model()
    checkpoint_callback = ModelCheckpoint("model.h5", monitor='loss', save_best_only=True, save_freq=2)
    callbacks = [checkpoint_callback, TuneReporterCallback()]
    model.compile(optimizer=Adam(lr=config["lr"]), loss=[dice_coef_loss], metrics = [dice_coef, 'binary_accuracy'])
    train_vol, train_seg, valid_vol, valid_seg = unet.DataLoader()
    loss_history = model.fit(x = train_vol, y = train_seg, batch_size = 16, epochs = 5, validation_data =(valid_vol, valid_seg), callbacks = callbacks)
    
    return loss_history

import numpy as np
import ray
np.random.seed(5)  
hyperparameter_space = {
    "lr": tune.loguniform(0.0002, 0.2)
    }

TRIALS = 3

ray.shutdown()  
ray.init(log_to_driver=False)
! rm -rf ~/ray_results/tune_unet

analysis = tune.run(
    tune_unet, 
    verbose=1, 
    config=hyperparameter_space,
    num_samples=TRIALS)

df = analysis.dataframe()

Trial name,status,loc,lr,loss,iter,total time (s),neg_mean_loss
tune_unet_d1b90_00000,TERMINATED,,0.00092685,-1.96922,5,638.149,1.96922
tune_unet_d1b90_00001,TERMINATED,,0.0818893,-1.96905,5,639.399,1.96905
tune_unet_d1b90_00002,TERMINATED,,0.000834041,-1.96922,5,627.955,1.96922


ValueError: No `metric` has been passed and  `default_metric` has not been set. Please specify the `metric` parameter.

In [85]:
df.to_csv("tune_unet.csv", sep='\t', encoding='utf-8')

In [87]:
df

,mean_accuracy,mean_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,neg_mean_loss,experiment_id,date,...,trial_id,experiment_tag,keras_info/loss,keras_info/dice_coef,keras_info/binary_accuracy,keras_info/val_loss,keras_info/val_dice_coef,keras_info/val_binary_accuracy,config/lr,logdir
0,NaN,-1.969369,156.057521,False,NaN,NaN,5,1.969369,0a46962a9fdf4a47bd947e3d93a42ef0,2020-10-01_04-49-26,...,480e8_00000,0_lr=0.00092685,-1.969369,1.969413,0.004293,-1.969349,1.969511,0.013983,0.000927,/home/scitech/ray_results/tune_unet/tune_unet_...
1,NaN,-1.969195,156.030143,False,NaN,NaN,5,1.969195,a50c42ed28544f93b603eac6616b6ca2,2020-10-01_04-49-23,...,480e8_00001,1_lr=0.081889,-1.969195,1.968757,0.000032,-1.968927,1.968831,0.000023,0.081889,/home/scitech/ray_results/tune_unet/tune_unet_...
2,NaN,-1.969351,156.931784,False,NaN,NaN,5,1.969351,fcee674f0996454eaca4585f9b4433e4,2020-10-01_04-49-29,...,480e8_00002,2_lr=0.00083404,-1.969351,1.969484,0.000380,-1.969180,1.969170,0.002304,0.000834,/home/scitech/ray_results/tune_unet/tune_unet_...
